In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    temperature=0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
)

In [2]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    model=model,
    tools=[],
    checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [HumanMessage("Hi! My name is Bob.")]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='4a555b38-5b3a-4933-881f-96e7a1bda10e'),
  AIMessage(content='Hi Bob! Nice to meet you. How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 15, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-2c309198-8ec1-4b34-b97d-bb6017d3d90c', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2d8f5627-e7b9-4247-8103-fee32cd17ad4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 17, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}

In [3]:
agent.invoke(
    {"messages": [HumanMessage("What's my name?")]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='4a555b38-5b3a-4933-881f-96e7a1bda10e'),
  AIMessage(content='Hi Bob! Nice to meet you. How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 15, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-2c309198-8ec1-4b34-b97d-bb6017d3d90c', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2d8f5627-e7b9-4247-8103-fee32cd17ad4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 17, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='eecc5a74-1871-4b66-9010-303a50aeec33'),
  AIMessage(content=

In [4]:
agent.invoke(
    {"messages": [HumanMessage("What's my name?")]},
    {"configurable": {"thread_id": "2"}},  
)

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='a4959c34-632f-43b8-886b-f9d56771dd77'),
  AIMessage(content='I don’t know your name! You haven’t told me yet. 😊  \nFeel free to share it if you’d like—I’d be happy to use it!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 13, 'total_tokens': 47, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-3793e603-bad5-4a5c-b772-28e3846064d1', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--193bb868-b5a0-4a16-bffc-9bf421a170e8-0', usage_metadata={'input_tokens': 13, 'output_tokens': 34, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}

In [6]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime
from langchain_core.runnables import RunnableConfig
from typing import Any


@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return None  # No changes needed

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model,
    tools=[],
    middleware=[trim_messages],
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

agent.invoke({"messages": HumanMessage("hi, my name is bob")}, config)
agent.invoke({"messages": HumanMessage("write a short poem about cats")}, config)
agent.invoke({"messages": HumanMessage("now do the same but for dogs")}, config)
final_response = agent.invoke({"messages": HumanMessage("what's my name?")}, config)

final_response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! 😊


In [7]:
final_response["messages"]

[HumanMessage(content='hi, my name is bob', additional_kwargs={}, response_metadata={}, id='2da7f4e7-7922-4826-b6b1-31418c3a50b0'),
 AIMessage(content="Here's a short poem for you, Bob:\n\n**Purr-fect Companions**\n\nSoft paws on the sill,  \nEyes like moonlight, calm and still.  \nA flick of the tail, a gentle sigh—  \nWatching clouds drift slowly by.  \n\nThen *pounce!* A toy, a sunbeam bright,  \nA blur of fur in playful flight.  \nBut when the day is done, you’ll see  \nThem curled up close, purring near me.  \n\n— For Bob, who appreciates cats! 😺", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 47, 'total_tokens': 159, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-0637c5ad-0939-4aa9-a97d-78d01b93b795', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e

In [8]:
from langchain.agents.middleware import after_model


@after_model
def delete_old_messages(state: AgentState, runtime: Runtime) -> dict | None:
    """Remove old messages to keep conversation manageable."""
    messages = state["messages"]
    if len(messages) > 2:
        # remove the earliest two messages
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
    return None


agent = create_agent(
    model=model,
    tools=[],
    system_prompt="Please be concise and to the point.",
    middleware=[delete_old_messages],
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

for event in agent.stream(
    {"messages": [HumanMessage("hi! I'm bob")]},
    config,
    stream_mode="values",
):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How can I help you today?')]


In [9]:
for event in agent.stream(
    {"messages": [HumanMessage("what's my name?")]},
    config,
    stream_mode="values",
):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How can I help you today?'), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob! How can I help you today?'), ('human', "what's my name?"), ('ai', 'Your name is Bob!')]
[('human', "what's my name?"), ('ai', 'Your name is Bob!')]


In [ ]:
from langchain.agents.middleware import SummarizationMiddleware


agent = create_agent(
    model=model,
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model=model,
            max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
        )
    ],
    checkpointer=InMemorySaver(),
)